In [1]:
%load_ext autoreload
%autoreload 2

In [2]:

import pyscisci.all as pyscisci

import numpy as np
import matplotlib.pylab as plt
try:
    import seaborn as sns
    sns.set_style('white')
except:
    pass

%matplotlib

Using matplotlib backend: MacOSX


In [3]:
# set this path to where the MAG database will be stored
path2openalex = '/home/ajgates/MAG'
path2openalex = '/Volumes/GatesExpansionDrive/DataSets/OpenAlex'


In [4]:
myoa = pyscisci.OpenAlex(path2openalex, keep_in_memory=False) 
# set keep_in_memory=False if you want to load the database each time its needed - good for when you 
# cant keep more than one database in memory at a time
# otherwise keep_in_memory=True will keep each database in memory after its loaded

In [5]:
df_features_to_keep = ['all']
myoa.download_from_source(rewrite_existing = False,
                         dataframe_list=df_features_to_keep)

In [22]:
myoa.parse_publications(show_progress=True, dflist='all')

Works:   0%|          | 0/160 [00:00<?, ?it/s]

True

In [ ]:
# before we can start running our analysis, we have to preprocess the raw data into
# DataFrames that are more convinent to work with

# we only need to run this for the first time, but it will take awhile
mymag.preprocess(dflist=['all'])

In [9]:
import boto3
from botocore import UNSIGNED
from botocore.config import Config
s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))

prefix=''
bucket='openalex'

base_kwargs = {
        'Bucket':bucket
    }
results = s3_client.list_objects_v2(**base_kwargs)

print(results)

{'ResponseMetadata': {'RequestId': 'TCS53AMVV2A0SPAE', 'HostId': '2/LL7thhdYfEH2vOxbpZlEIzT0TeMCg5QXd2E/7BKNUPBrfgEPG6bdfbf64XVBmYEPVmkX8K02o=', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amz-id-2': '2/LL7thhdYfEH2vOxbpZlEIzT0TeMCg5QXd2E/7BKNUPBrfgEPG6bdfbf64XVBmYEPVmkX8K02o=', 'x-amz-request-id': 'TCS53AMVV2A0SPAE', 'date': 'Wed, 12 Jan 2022 18:55:08 GMT', 'x-amz-bucket-region': 'us-east-1', 'content-type': 'application/xml', 'transfer-encoding': 'chunked', 'server': 'AmazonS3'}, 'RetryAttempts': 0}, 'IsTruncated': False, 'Contents': [{'Key': 'LICENSE.txt', 'LastModified': datetime.datetime(2022, 1, 2, 19, 26, 34, tzinfo=tzutc()), 'ETag': '"3853e2a78a247145b4aa16667736f6de"', 'Size': 7047, 'StorageClass': 'STANDARD'}, {'Key': 'README.txt', 'LastModified': datetime.datetime(2022, 1, 3, 18, 32, 18, tzinfo=tzutc()), 'ETag': '"1b1b3325240d266e70fcca45a7787b39"', 'Size': 301, 'StorageClass': 'STANDARD'}, {'Key': 'RELEASE_NOTES.txt', 'LastModified': datetime.datetime(2022, 1, 2, 19, 26, 34, 

In [11]:
[k for k in results.keys()]

['ResponseMetadata',
 'IsTruncated',
 'Contents',
 'Name',
 'Prefix',
 'MaxKeys',
 'EncodingType',
 'KeyCount']

In [14]:
set([k['Key'].split('/')[1] for k in results['Contents'] if 'data' in k['Key']])

{'authors', 'concepts', 'institutions', 'venues', 'works'}

In [15]:
sorted([k['Key'] for k in results['Contents']])

['LICENSE.txt',
 'README.txt',
 'RELEASE_NOTES.txt',
 'data/authors/manifest',
 'data/authors/updated_date=2021-12-30/0000_part_00.gz',
 'data/authors/updated_date=2021-12-30/0001_part_00.gz',
 'data/authors/updated_date=2021-12-30/0002_part_00.gz',
 'data/authors/updated_date=2021-12-30/0003_part_00.gz',
 'data/authors/updated_date=2021-12-30/0004_part_00.gz',
 'data/authors/updated_date=2021-12-30/0005_part_00.gz',
 'data/authors/updated_date=2021-12-30/0006_part_00.gz',
 'data/authors/updated_date=2021-12-30/0007_part_00.gz',
 'data/authors/updated_date=2021-12-30/0008_part_00.gz',
 'data/authors/updated_date=2021-12-30/0009_part_00.gz',
 'data/authors/updated_date=2021-12-30/0010_part_00.gz',
 'data/authors/updated_date=2021-12-30/0011_part_00.gz',
 'data/authors/updated_date=2021-12-30/0012_part_00.gz',
 'data/authors/updated_date=2021-12-30/0013_part_00.gz',
 'data/authors/updated_date=2021-12-30/0014_part_00.gz',
 'data/authors/updated_date=2021-12-30/0015_part_00.gz',
 'data/au

In [16]:
s3_client.download_file('openalex', 
                        'data/works/updated_date=2022-01-02/0000_part_00.gz', 
                        '/Users/ajgates/Desktop/0000_part_00.gz')

In [23]:
import gzip
import json

dest_pathname = '/Users/ajgates/Desktop/0000_part_00.gz'
with gzip.open(dest_pathname, 'r') as infile:
    with gzip.open(dest_pathname.split('.')[0] + '_temp.gz', 'w') as outfile:
        for line in infile:
            #print(line)
            jline = json.loads(line)
            #print(jline)
            newline = json.dumps(jline) + "\n"
            outfile.write(newline.encode('utf-8'))
            